In [ ]:
# Подключение модулей
from keras.models import Sequential
from keras.layers.core import Activation, Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from keras.applications.xception import Xception, preprocess_input
import cv2, numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Пути к наборам данных
train_dir = '/content/drive/MyDrive/horses_or_humans/train'
test_dir = '/content/drive/MyDrive/horses_or_humans/val' 
# Размер изображений
img_width, img_height = 300, 300
# Размер мини-выборки
batch_size = 64
# Кол-во изображений для обучения
nb_train_samples = 1027
# Кол-во изображений для теста
nb_test_samples = 256

In [ ]:
# Создание генератора изображений
datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
# Создание экземпляра модели сети VGG16
xception = Xception(
    weights='imagenet', 
    include_top=False, 
    input_shape=(img_width, img_height, 3))
# weights - веса предварительно обученной сети
# include_top = false означает, что мы загружаем только сверточную часть сети, без квалификационной
# input_shape - размер тензора

# Сверточную часть сети обучать не надо
xception.trainable = False

xception.summary()

In [ ]:
# Создание модели составной сети
model = Sequential()
# Добавляем сверточные слои
model.add(vgg16_net)
# Преобразуем двумерный массив vgg16 в одномерный
model.add(Flatten())
# Полносвязный слой
model.add(Dense(256, activation='relu'))
# Слой регуляризации (для предотвращения переобучения)
model.add(Dropout(0.5))
# Кол-во классов
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='SGD', 
              metrics=['accuracy'])

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=5,
    validation_data=test_generator,
    validation_steps=nb_test_samples // batch_size)

In [ ]:
# Проверяем модель на тестовой выборке
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print('Точность работы на тестовых данных: %.2f%%' % (scores[1]*100))